In [1]:
import pyomo.environ as pyo

Problema de la dieta: Saber cómo alimentar a los soldados de manera que pudieran tener la cantidad de nutrientes necesarios en las comidas, logrando menor costo posible.

## Set

$F$ = Set of foods

$N$ = Set of nutrients

## Variables

$x_{f}$ = Number of serving of foof $f$

## Parameters

$c_{f}$ = Cost per serving of food $f$, $\forall f \in F$

$a_{f,n}$ = Amount of nutrients $n$ in food $f$, $\forall f \in F, n \in N $ 

$Nmin_{n}$ = Minimum level of nutrient $n$, $\forall n \in N$

$Nmax_{n}$ = Maximum level of nutrient $n$, $\forall n \in N$

$V_{f}$ = The volume per serving of food $f$, $\forall f \in F$

$Vmax$ = Maximum volume of food consumed

## Objective

Minimize the total cost of foods

$$\text{min} \sum c_f x_f$$

## Constraints

1. Limit nutrient consuption for each nutrient

$$ Nmin\leq \sum a_{f,n} \cdot x_f \leq Nmax_{n} , \forall n \in N$$

2. Limit the volume of food consumed

$$ \sum x_f \cdot V_f \leq Vmax, \forall f \in F$$

3. Consumption lower bound

$$ x_f \geq 0$$

# Pyomo modelling concrete

In [2]:
import pyomo.environ as pyo

In [3]:
model = pyo.ConcreteModel(name="Diet problem", doc="Diet problem")

In [4]:
FOODS = {"Cheeseburger", "Fries", "Coke"}
NUTRIENTS = {"Calories", "Carbs", "Proteins", "Sugars"}

In [5]:
model.F = pyo.Set(initialize=FOODS, doc = "Food set")
model.N = pyo.Set(initialize=NUTRIENTS, doc = "Nutrients set")

(type: set).  This WILL potentially lead to nondeterministic behavior in Pyomo
(type: set).  This WILL potentially lead to nondeterministic behavior in Pyomo


In [6]:
cf = {
    "Cheeseburger": 1.84,
    "Fries":0.67,
    "Coke":1,
}

afn = {
    ("Cheeseburger", "Calories"): 510,
    ("Cheeseburger", "Carbs"): 34,
    ("Cheeseburger", "Proteins"): 28,
    ("Cheeseburger", "Sugars"): 5,
    ("Fries", "Calories"): 220,
    ("Fries", "Carbs"): 26,
    ("Fries", "Proteins"): 3,
    ("Fries", "Sugars"): 2,
    ("Coke", "Calories"): 120,
    ("Coke", "Carbs"): 10,
    ("Coke", "Proteins"): 1,
    ("Coke", "Sugars"): 15,
}

Nminn = {
    "Calories":2000,
    "Carbs":120,
    "Proteins":50,
    "Sugars":44,
}

Nmaxn = {
    "Calories":1E10,
    "Carbs":1E10,
    "Proteins":1E10,
    "Sugars":1E10,
}

Vmax = 70


Vf = {
    "Cheeseburger":5, 
    "Fries":5, 
    "Coke":5,
}


model.cf= pyo.Param(model.F, within=pyo.NonNegativeReals, initialize=cf, doc="cost per serving")
model.afn = pyo.Param(model.F, model.N, initialize=afn)
model.Nminn = pyo.Param(model.N, initialize=Nminn, doc="Minimum nutrients per serving")
model.Nmaxn = pyo.Param(model.N, initialize=Nmaxn, doc="Maximun nutrients per serving")
model.Vmax = pyo.Param(initialize=Vmax)
model.Vf = pyo.Param(model.F, initialize=Vf, doc="Volume consumption per saving")

In [7]:
model.xf = pyo.Var(model.F, domain=pyo.NonNegativeReals, doc="Amount of food serving")

In [8]:
#objective 

def cost_function(model):
    return sum( model.cf[f]*model.xf[f] for f in model.F)

model.objective = pyo.Objective(rule=cost_function, sense=pyo.minimize)

In [9]:
#Constraints
# 1. Nutrient limits
def nutrient_limit_min(model, n):
   return model.Nminn[n] <= sum(model.afn[f,n]*model.xf[f] for f in model.F)

def nutrient_limit_max(model, n):
   return sum(model.afn[f,n]*model.xf[f] for f in model.F) <= model.Nmaxn[n]


model.nutrient_constraint_min = pyo.Constraint(model.N, rule=nutrient_limit_min, doc="nutrient limit constraint min")
model.nutrient_constraint_max = pyo.Constraint(model.N, rule=nutrient_limit_max, doc="nutrient limit constraint max")



In [10]:
# 2. Volume maximum
def max_rule(model):
    return sum(model.xf[f]*model.Vf[f] for f in model.F) <= model.Vmax

model.volume_contraint= pyo.Constraint(rule=max_rule, doc="volume constraint" )

## Solve the model

In [11]:
solver = pyo.SolverFactory("appsi_highs")

In [12]:
solver.solve(model, tee=True)

Running HiGHS 1.5.3 [date: 2023-05-16, git hash: 594fa5a9d]
Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
9 rows, 3 cols, 27 nonzeros
5 rows, 3 cols, 15 nonzeros
Presolve : Reductions: rows 5(-4); columns 3(-0); elements 15(-12)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Pr: 4(533) 0s
          3     7.5755065943e+00 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 3
Objective value     :  7.5755065943e+00
HiGHS run time      :          0.00


{'Problem': [{'Lower bound': 7.5755065942591155, 'Upper bound': 7.5755065942591155, 'Number of objectives': 1, 'Number of constraints': 0, 'Number of variables': 0, 'Sense': 1}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Termination message': 'TerminationCondition.optimal'}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [13]:
model.objective.expr()

7.5755065942591155

In [14]:
model.xf.display()

xf : Amount of food serving
    Size=3, Index=F
    Key          : Lower : Value             : Upper : Fixed : Stale : Domain
    Cheeseburger :     0 : 1.135143522110163 :  None : False : False : NonNegativeReals
            Coke :     0 : 1.826532195500388 :  None : False : False : NonNegativeReals
           Fries :     0 : 5.463149728471683 :  None : False : False : NonNegativeReals


In [15]:
model.nutrient_constraint_min.display()

nutrient_constraint_min : Size=4
    Key      : Lower  : Body               : Upper
    Calories : 2000.0 : 1999.9999999999998 :  None
       Carbs :  120.0 : 198.90209464701317 :  None
    Proteins :   50.0 :               50.0 :  None
      Sugars :   44.0 :  43.99999999999999 :  None


In [16]:
model.nutrient_constraint_max.display()

nutrient_constraint_max : Size=4
    Key      : Lower : Body               : Upper
    Calories :  None : 1999.9999999999998 : 10000000000.0
       Carbs :  None : 198.90209464701317 : 10000000000.0
    Proteins :  None :               50.0 : 10000000000.0
      Sugars :  None :  43.99999999999999 : 10000000000.0


In [17]:
model.pprint()

Diet problem

    3 Set Declarations
        F : Food set
            Size=1, Index=None, Ordered=Insertion
            Key  : Dimen : Domain : Size : Members
            None :     1 :    Any :    3 : {'Coke', 'Fries', 'Cheeseburger'}
        N : Nutrients set
            Size=1, Index=None, Ordered=Insertion
            Key  : Dimen : Domain : Size : Members
            None :     1 :    Any :    4 : {'Proteins', 'Calories', 'Carbs', 'Sugars'}
        afn_index : Size=1, Index=None, Ordered=True
            Key  : Dimen : Domain : Size : Members
            None :     2 :    F*N :   12 : {('Coke', 'Proteins'), ('Coke', 'Calories'), ('Coke', 'Carbs'), ('Coke', 'Sugars'), ('Fries', 'Proteins'), ('Fries', 'Calories'), ('Fries', 'Carbs'), ('Fries', 'Sugars'), ('Cheeseburger', 'Proteins'), ('Cheeseburger', 'Calories'), ('Cheeseburger', 'Carbs'), ('Cheeseburger', 'Sugars')}

    6 Param Declarations
        Nmaxn : Maximun nutrients per serving
            Size=4, Index=N, Domain=Any, Defa

In [26]:
import pandas as pd

In [27]:
data = pd.DataFrame(columns=["Food", "Calories", "Carbs", "Proteins", "Sugars", "Cost", "Volume"])

In [28]:
for f in FOODS:
    calories = afn[(f, "Calories")]
    carbs = afn[(f, "Carbs")]
    proteins = afn[(f, "Proteins")]
    sugars = afn[(f, "Sugars")]
    cost = cf[f]
    volume = Vf[f]
    data = data.append(
        {
            "Food": f,
            "Calories": calories,
            "Carbs": carbs,
            "Proteins": proteins,
            "Sugars": sugars,
            "Cost": cost,
            "Volume": volume,
        },
        ignore_index=True,
    )

/var/folders/jz/p2352pjj3qlbg227tsvh5vwc0000gn/T/ipykernel_86904/3462708485.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(
/var/folders/jz/p2352pjj3qlbg227tsvh5vwc0000gn/T/ipykernel_86904/3462708485.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(
/var/folders/jz/p2352pjj3qlbg227tsvh5vwc0000gn/T/ipykernel_86904/3462708485.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(


In [30]:
data

,Food,Calories,Carbs,Proteins,Sugars,Cost,Volume
0,Coke,120,10,1,15,1,5
1,Fries,220,26,3,2,0.67,5
2,Cheeseburger,510,34,28,5,1.84,5
